### Regresión lineal 
El dataset en cuestión es 'taxi_trip_pricingps' que contiene información sobre las tarifas de los viajes en taxi, sin embargo, en este ejemplo se desarrollará un modelo lineal que su ajuste sea sobre una variable del conjunto de forma que se cumplan requisitos entre ambas variables.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from scipy.stats import pearsonr, zscore, kurtosis, norm, shapiro
from sklearn.linear_model import LinearRegression


df = pd.read_csv("data/taxi_trip_pricing.csv")
df.dropna(subset=["Trip_Distance_km","Trip_Price"], inplace=True) # eliminación de filas nulas

df

#### Análisis Descriptivo
Si bien definos usar unicamente dos variables para generar regresiones, en la siguiente celda se describiren las variables cualitativas sobre las continuas, mencionando las siguientes características:
- En todas variables hay un claro caso de valores que se alejan de la media de forma creciente 
- Durante los días de semana en la mañana y sobretodo en el día, se producen los mayores viajes
- En la mayorías de los viajes el tráfico suele ser bajo o mediano y el número de pasajeros no suele variar

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(14,12))

df["Passenger_Count"].fillna("Desconocido", inplace=True)
df["Passenger_Count"] = df["Passenger_Count"].astype(str)

sns.kdeplot(df,x="Trip_Price",hue="Time_of_Day",ax=ax[0,0])
sns.kdeplot(df,x="Trip_Price",hue="Day_of_Week",ax=ax[0,1])
sns.kdeplot(df,x="Trip_Price",hue="Traffic_Conditions",ax=ax[1,0])
sns.kdeplot(df,x="Trip_Price",hue="Passenger_Count",ax=ax[1,1])
ax[0,0].grid("on")
ax[0,1].grid("on")
ax[1,0].grid("on")
ax[1,1].grid("on")

plt.show()

#### Análisis Exploratorio de datos

Son un conjunto de métodos que utilizan conceptos estadísticos enfocados en explorar y analizar el comportamiento de los datos donde el objetivo es descubrir patrones, relaciones y estructuras que guíen la implementación de un modelo particular como solución. Algunos puntos clave en un caso de regresión son:

Distribución Normal: su presencia en las variables continuas es fundamentalmente requerida para algunos modelos regresores debido a la estructura que genera a la hora de ajustarse a los datos, existen diferentes formas de estimar una distribución de este tipo, como medias y medianas que coinciden, representaciones gráficas con forma simétrica respecto a la media y acampanada con la mayoría de los valores concetrados, sin colas pesadas o nulas(curtosis mesocúritca, cercana a 0).

Correlación Lineal: valor estadístico que indica la fuerza y dirección de una asociación lineal entre dos variables númericas, es una de las medidas estadísticas más importantes y una correlación lineal puede llegar a explicar la influecia de una variable independiente sobre una dependiente.

Outliers: valores que destacan por su notoria distinción(mayor o menor) a los demás del conjunto. La forma más efectiva de hallar estos últimos es la Regla Empírica, según la cual los datos que se encuentran dentro de 3 desviaciones éstandar con respecto a la media representan el 99.7% de los datos, por lo que podemos concluir con bastante seguridad que los datos cuyos valores Z caen por afuera de este umbral son atípicos, ya que son distintos al 99.7%.


In [ ]:
corr, _ = pearsonr(df["Trip_Distance_km"], df["Trip_Price"])

var_x = df["Trip_Distance_km"].var()
var_y = df["Trip_Price"].var()

df_zscore = df[["Trip_Distance_km","Trip_Price"]]

df_zscore["Trip_Distance_km_zscore"] = zscore(df["Trip_Distance_km"]).abs()
df_zscore["Trip_Price_zscore"] = zscore(df["Trip_Price"]).abs()

outliers = df_zscore.loc[(df_zscore["Trip_Distance_km_zscore"] > 3) | (df_zscore["Trip_Price_zscore"] > 3),["Trip_Distance_km","Trip_Price"]]

curtosis_x = kurtosis(df["Trip_Distance_km"])
curtosis_y = kurtosis(df["Trip_Price"])

print("--------------------------------------------------------------------------------------------------")
print(f"Correlación Pearsonr: {round(corr,2)}")
print(f"Varianza variable X: {round(var_x,2)} | Varianza variable Y: {round(var_y,2)}")
print(f"Outliers: {outliers.shape[0]}")
print(f"Curtosis variable X: {round(curtosis_x,1)} | Curtosis variable Y: {round(curtosis_y,1)}")
print("--------------------------------------------------------------------------------------------------")

#### Pruebas de normalidad
Los métodos estadísticos mencionados anteriormente son ampliamente utilizados y dan una base sólida para diagnosticar, sin embargo, las siguientes pruebas de hipótesis proporcionan una evaluación más precisa y son menos propensas a errores de confirmación

Shapiro-Wilk: Se basa en la correlación entre los valores del conjunto y una versión de estos que representan un respectivo de nivel de desviación éstandar, que se denominan valores Z o estandarizados.

Kolmogorov-Smirnov: Compara la función de distribución acumulativa(CDF) del conjunto de valores con la de una distribución normal.

Ambas pruebas hacen uso del valor P para concluir su hipótesis nula(los datos siguen una distribución normal asimilada) o su hipótesis alternativa(caso contrario) con un valor P muy cercano, igual o menor a 0.05.

In [ ]:
df["Trip_Distance_km_zscore"] = zscore(df["Trip_Distance_km"])
df["Trip_Price_zscore"] = zscore(df["Trip_Price"])

cdf_Trip_Distance_km = norm.cdf(df["Trip_Distance_km"], df["Trip_Distance_km"].mean(), df["Trip_Distance_km"].std()) 
cdf_Trip_Price = norm.cdf(df["Trip_Price"], df["Trip_Price"].mean(), df["Trip_Price"].std()) 

fig = make_subplots(rows=2, cols=2)

fig.add_trace(go.Scatter(x=df["Trip_Distance_km_zscore"], y=df["Trip_Distance_km"], mode="markers", name="KM recorridos"), row=1,col=1)
fig.add_trace(go.Scatter(x=df["Trip_Price_zscore"], y=df["Trip_Price"], mode="markers", name="Tarifa total"), row=1,col=2)
fig.add_trace(go.Scatter(x=df["Trip_Distance_km"].values, y=cdf_Trip_Distance_km, mode="markers", name="KM recorridos"), row=2,col=1)
fig.add_trace(go.Scatter(x=df["Trip_Price"].values, y=cdf_Trip_Price, mode="markers", name="Tarifa total"), row=2,col=2)
fig.update_layout(height=600, width=850, title_text="Shapiro-wilk & Kolmogorov-smirnov")

_, p_value_var_x = shapiro(df["Trip_Distance_km"])
_, p_value_var_y = shapiro(df["Trip_Price"])

fig

#### Modelo lineal

Luego de haber obtenido los distintos resultados de las variables seleccionadas para la estimación de tarifas en viajes de Taxis como una correlación igual 0.85, una distribución asimétrica sesgada positivamente y ciertos valores atípicos se procede a generar el modelo lineal y ponerlo en práctica.

In [50]:
# eliminación de valores atípicos
df = df_zscore.loc[(df_zscore["Trip_Distance_km_zscore"] < 3) | (df_zscore["Trip_Price_zscore"] < 3),:]

linear_regression = LinearRegression()
 
var_x = df["Trip_Distance_km"].values.reshape((-1,1)) # variable X en la correlación lineal
var_y = df["Trip_Price"] # variable Y en la correlación lineal

linear_regression.fit(var_x, var_y)

# regresión de nuevos objetos aleatorios

objects = np.array([[28.15],[12.5],[3.8],[8.25],[19.5],[32.7],[40.9],[45]])

predicts = linear_regression.predict(objects)

#### Dashboard que refleja los insigths obtenidos y el ajuste del modelo lineal sobre los datos

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e2_body",children=[
    html.H1("Tarifas de taxis",id="title",className="e2_title"),
    html.Div(id="dashboard",className="e2_dashboard",children=[
        html.Div(id="column-1",className="e2_column_1",children=[
            dcc.Dropdown(id="dropdown",className="e2_dropdown",
                        options=[
                            {"label":"Kilometros recorridos","value":"Trip_Distance_km"},
                            {"label":"Tarifas","value":"Trip_Price"}
                        ],
                        value="Trip_Distance_km",
                        multi=False,
                        clearable=False),
            html.Div(className="e2_div_graphs",children=[
                dcc.Graph(id="graph-1",className="e2_graphs",figure={}), 
                dcc.Graph(id="graph-2",className="e2_graphs",figure={})
            ])
        ]),
        html.Div(id="column-2",className="e2_column_2",children=[
            html.H2("Shapiro-Wilk Valor-P",id="p_values_title",className="e2_p_values_title"),
            html.Div(id="p_values",className="e2_stats_div",children=[
                html.Div(id="p_value_var_x",className="e2_stats",children=[html.P(f"Variable X: {round(p_value_var_x,2)}",style={"font-size":"1em"})]),
                html.Div(id="p_value_var_y",className="e2_stats",children=[html.P(f"Variable Y: {round(p_value_var_y,2)}",style={"font-size":"0.98em"})])
            ]),
            html.Div(f"Correlación: {round(corr,2)}",className="e2_corr",id="corr"),
            dcc.Graph(id="graph-3",className="e2_graph_3",figure={})
        ])
    ])
])

@app.callback(
    [Output(component_id="graph-1",component_property="figure"),
    Output(component_id="graph-2",component_property="figure"),
    Output(component_id="graph-3",component_property="figure")],
    [Input(component_id="dropdown",component_property="value")]
)

def update_dash(slct_var):
    
    mean = df[slct_var].mean()
    median = df[slct_var].median()
    
    extr_list = [0]
    
    var_title = "Kilometros recorridos"
    
    if slct_var == "Trip_Price":
        extr_list.append(60)
        var_title = "Tarifas"
    elif slct_var == "Trip_Distance_km":
        extr_list.append(40)
        var_title = "Kilometros recorridos"
    
    histplot = go.Figure(go.Histogram(x=df[slct_var],name="Distribución"))
    histplot.add_trace(go.Scatter(x=[mean,mean],y=extr_list,mode="lines+markers",marker_color="red",name="Media"))
    histplot.add_trace(go.Scatter(x=[median,median],y=extr_list,mode="lines+markers",marker_color="green",name="Mediana"))
    histplot.update_layout(title="Histograma",xaxis_title=var_title)
    
    boxplot = px.box(df, y=slct_var, points="all", title="Gráfico de caja")
    
    scatter = go.Figure()
    scatter.add_trace(go.Scatter(x=df["Trip_Distance_km"],y=df["Trip_Price"],mode="markers",marker_color="blue",name="Tarifas reales"))
    scatter.add_trace(go.Scatter(x=objects.reshape(-1),y=predicts,mode="lines+markers",marker_color="red",name="Predicciones"))
    scatter.update_layout(title="Regresión Lineal",xaxis_title="Kilometros recorridos",yaxis_title="Tarifas")

    return histplot, boxplot, scatter

if __name__ == "__main__":
    app.run_server(debug=False)

#### Error de bías
También denominado sesgo, es la diferencia entre la suposiciones asumidas por el modelo y los valores verdaderos. Sucede principalmente en algoritmos paramétricos que requieren de ciertas estructuras o relaciones en los datos para una mejor interpretación. Alto bías: requiere más suposiciones a la hora de estimar la función objetivo, ejemplos de algoritmos con alto bías: Regresión Lineal, Regresión Logística, Naive Bayes, algoritmos de series temporales.

Este ejemplo que plantea un problema cotidiano donde se cree que la distancia recorrida engloba la totalidad de la estimación de la tarifa en un viaje sirve como explicación de uno de los principales desafíos presentes en una resolución y que lleva a resultados erróneos. El algoritmo de Regresión lineal asume principalmente que el incremento o decremento de una variable influye a otra, teniendo en cuenta su suposición, la distorsión sistemática en el patrón lineal de los datos que difieren notoriamente del resto llega a afectar la dirección de los resultados, en este caso en particular no había escasa información sino que al contrario, de forma adrede, se usó solo el primer factor mencionado y aparte se analizaron a las etiquetas restantes del conjunto, que pueden explicar algunos factores, por ejemplo: el exceso de tráfico afectó el incremento de la tarifa provocado por el horario y el día de semana. 
Entonces, quiero dejar como conclusión e indicarlo como uno de los conceptos más importantes que aprendí, es que no solo basta con eliminar los valores atípicos o utilizar modelos más robustos, sino que se debe resolver y prever el origen del sesgo para haber resuelto de manera completa el probelma en cuestión.  